In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

# Generating Data

In [2]:
def generate_date(year_1 = 1980, year_2 = 2000):
    return datetime.datetime(year_1, 1, 1) + datetime.timedelta(days= np.random.randint((datetime.datetime(year_2, 12,31) - datetime.datetime(year_1, 1,1)).days))  

In [3]:
emp_counts = int(1e3)
# Creating DataFrame
data = pd.DataFrame(data = {'id' : np.random.randint(0, emp_counts*10, emp_counts), 
                            'gender' : ['Male' if np.random.rand()<0.5 else 'Female' for i in range(emp_counts)],
                            'dob' : [generate_date() for i in range(emp_counts)],
                            'doh' : [generate_date(year_1 = 2008, year_2 = 2023) for i in range(emp_counts)],
                            'salary' : 6e6+ np.random.uniform(low = -3e6, high = 10e6, size = emp_counts) },)
data['dob'] = pd.to_datetime(data['dob'])
data['doh'] = pd.to_datetime(data['doh'])
data

,id,gender,dob,doh,salary
0,8983,Female,1980-08-07,2014-09-28,1.389172e+07
1,9290,Male,1983-06-17,2008-05-26,3.350022e+06
2,1343,Female,1997-11-26,2010-10-28,1.599512e+07
3,408,Female,1997-01-18,2022-03-30,1.276949e+07
4,7357,Female,1988-02-19,2009-03-01,1.488424e+07
...,...,...,...,...,...
995,8053,Male,1989-12-30,2010-05-03,1.572086e+07
996,2775,Female,1996-09-05,2008-07-03,6.427697e+06
997,398,Female,1995-07-13,2012-07-23,1.390129e+07
998,2464,Male,1986-04-28,2008-02-12,6.460066e+06


In [4]:
val_date = pd.Timestamp('2023-12-31')
data['age'] = np.round((val_date- data.dob)/np.timedelta64(1, 'Y'),2)
data['yos'] = np.round((val_date- data.doh)/np.timedelta64(1, 'Y'),2)

# Actuarial Assumptions

# Demographic Assumptions

Assume death probability follows 4th Indonesia Mortality Table. Disability probability is 1% of the former mortality table and resignation rate is 1% decreasing linearly from age 22 to age 56 (pension age).

In [5]:
mortality_base = pd.read_csv(r'data/TMI IV.csv')
pension_age = 56

In [6]:
def resignation_rate(entry_age, start_age = 22, end_age = pension_age, start_rate = 0.01, end_rate = 0):
    return start_rate +(end_rate - start_rate)*(np.arange(entry_age, end_age) - start_age)/(end_age - start_age) 

In [7]:
def demographic_table(table = mortality_base, employee_gender = None, employee_age = None, pension_age = pension_age):
    death = mortality_base[employee_gender].loc[int(employee_age):pension_age-1]
    disable = death*0.01
    resign = resignation_rate(entry_age = employee_age)
    return pd.DataFrame(data = {'death': death, 'disable': disable.values, 'resign' : resign}, 
                        index = np.arange(np.floor(employee_age), pension_age))
    

In [8]:
demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]).shape

(13, 3)

In [9]:
def service_table(demographic_tbl):
    survive = np.ones((demographic_tbl.shape[0], 4))
    for i in range(demographic_tbl.shape[0]):
        survive[i,1] = survive[i,0] * demographic_tbl.iloc[i,0] * (1 - demographic_tbl.iloc[i,1]) * (1 - demographic_tbl.iloc[i,2])
        survive[i,2] = survive[i,0] * demographic_tbl.iloc[i,1] * (1 - demographic_tbl.iloc[i,0]) * (1 - demographic_tbl.iloc[i,2])
        survive[i,3] = survive[i,0] * demographic_tbl.iloc[i,2] * (1 - demographic_tbl.iloc[i,0]) * (1 - demographic_tbl.iloc[i,1])
        try : 
            survive[i+1,0] = survive[i,0] - np.sum(survive[i,1:])  
        except : 
            survive = np.append(survive, np.array([[survive[i,0] - np.sum(survive[i,1:]), 0, 0, 0]]), axis = 0)
    return pd.DataFrame(data = survive, columns = ['survive', 'death', 'disable', 'resign'], index = np.append(demographic_tbl.index, 56))

In [10]:
service_table(demographic_tbl = demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]))

,survive,death,disable,resign
43.0,1.000000,0.001534,0.000015,0.003691
44.0,0.994759,0.001675,0.000017,0.003379
45.0,0.989688,0.001845,0.000018,0.003071
46.0,0.984753,0.002052,0.000020,0.002766
47.0,0.979915,0.002248,0.000022,0.002464
48.0,0.975180,0.002462,0.000025,0.002166
49.0,0.970528,0.002683,0.000027,0.001870
50.0,0.965948,0.002941,0.000029,0.001578
51.0,0.961400,0.003216,0.000032,0.001288
52.0,0.956864,0.003517,0.000035,0.001001


## Financial Assumption

In [11]:
sev_svc = pd.DataFrame({'severance': [min(i+1,9) for i in range(60)],
                        'service' : [0,0,0,2,2,2,3,3,3,
                                     4,4,4,5,5,5,6,6,6,
                                     7,7,7,8,8,8,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,
                                     10,10,10,10,10,10,10,10,10,10,10,10,10,10,10]})
ben_fac = pd.DataFrame({'retire': 1.75*sev_svc['severance']+sev_svc['service'],
                        'death': 2*sev_svc['severance']+sev_svc['service'],
                        'disable': 2*sev_svc['severance']+sev_svc['service'],
                        'resign': [1]*sev_svc.shape[0]})
ben_fac

,retire,death,disable,resign
0,1.75,2,2,1
1,3.50,4,4,1
2,5.25,6,6,1
3,9.00,10,10,1
4,10.75,12,12,1
5,12.50,14,14,1
6,15.25,17,17,1
7,17.00,19,19,1
8,18.75,21,21,1
9,19.75,22,22,1


In [12]:
def select_ben_fac(employee_age, employee_yos, pension_age, ben_fac = ben_fac):
    return ben_fac.iloc[int(employee_yos):int(employee_yos+pension_age-employee_age)+1, 1:]

In [13]:
select_ben_fac(employee_age=data.age.iloc[0], employee_yos= data.yos.iloc[0],pension_age=pension_age)

,death,disable,resign
9,22,22,1
10,22,22,1
11,22,22,1
12,23,23,1
13,23,23,1
14,23,23,1
15,24,24,1
16,24,24,1
17,24,24,1
18,25,25,1


### Discount Rate

In [14]:
yield_curve  = pd.read_csv(r'data/YieldCurve1Sep.csv')
yield_curve

,enor Year,Today
0,0.1,0.061580
1,1.0,0.061591
2,2.0,0.061629
3,3.0,0.061763
4,4.0,0.062007
5,5.0,0.062345
6,6.0,0.062754
7,7.0,0.063204
8,8.0,0.063671
9,9.0,0.064134


In [15]:
def spot_rates(yield_curve):
    spot_rate = np.zeros(yield_curve.shape[0])
    t = yield_curve.iloc[:,0]
    spot_rate[0] = yield_curve.iloc[0,1]
    spot_rate[1] = yield_curve.iloc[1,1]
    for i in range(2,yield_curve.shape[0]):
        sum = 0
        for j in range(1,i):
            sum += yield_curve.iloc[j,1]/(1+spot_rate[j])**t[j]
        spot_rate[i] = ((1+yield_curve.iloc[i,1])/(1-sum))**(1/t[i]) - 1   
    return pd.DataFrame(data = {'spot_rate' : spot_rate})

In [16]:
def discount_factor(employee_age, pension_age, rate, type = 'multi-rate'):
    if type == 'multi-rate':
        pass
    elif type == 'single-rate':
        rate = pd.DataFrame([rate]*(pension_age-int(employee_age)))
    else :
        raise 'Please define type of rate that being used either "multi-rate" or "single-rate"'
    return np.append(np.array([(1+rate.iloc[min(i,30)][0])**-i for i in range(pension_age-int(employee_age))]),
        (1+rate.iloc[min(30,pension_age-int(employee_age)-1)][0])**-(pension_age-employee_age))

In [17]:
discount_factor(employee_age= data.age.iloc[0], pension_age = pension_age ,rate= spot_rates(yield_curve), type ='multi-rate')

array([1.        , 0.94198236, 0.88729901, 0.83568462, 0.78689182,
       0.74071213, 0.69697421, 0.65554137, 0.61630088, 0.57915721,
       0.54402464, 0.51082366, 0.4794778 , 0.4630256 ])

### Salary Increase 

In [18]:
salary_inc = 0.05
def salary_factor(employee_age, pension_age, salary_inc = salary_inc) :
    return np.append(np.array([(1+salary_inc)**i for i in range(pension_age - int(employee_age))]),
    (1+salary_inc)**(pension_age - employee_age))

In [19]:
salary_factor(employee_age= data.age.iloc[0], pension_age = pension_age)

array([1.        , 1.05      , 1.1025    , 1.157625  , 1.21550625,
       1.27628156, 1.34009564, 1.40710042, 1.47745544, 1.55132822,
       1.62889463, 1.71033936, 1.79585633, 1.84650075])

### Defined Benefit Obligation Factor

In [20]:
def dbo_factor(employee_age, employee_yos, pension_age = pension_age):
    return np.array([employee_yos/(employee_yos+i) for i in range(pension_age-int(employee_age))])

In [21]:
dbo_factor(employee_age=data.age.iloc[0], employee_yos=data.yos.iloc[0])

array([1.        , 0.90253411, 0.82238011, 0.75530179, 0.69834087,
       0.64936886, 0.6068152 , 0.56949569, 0.53650058, 0.50711939,
       0.4807892 , 0.45705824, 0.43555974])

# Present Value Benefit

## Combining All Financial and Demographic Assumptions Excluding Pension Calculation

This approach can be rearranged if there is tax gross up. If there is tax gross up, flop will increase at least $n$ times with $2n$ with $n$ is 55-$\lfloor \text{entry age}\rfloor$

In [22]:
financial_assumption = np.multiply(np.multiply(discount_factor(employee_age= data.age.iloc[0], pension_age = pension_age ,rate= spot_rates(yield_curve), type ='multi-rate'),
            salary_factor(employee_age= data.age.iloc[0], pension_age = pension_age),
            ),np.append(dbo_factor(employee_age=data.age.iloc[0], employee_yos=data.yos.iloc[0]),1))
financial_assumption

array([1.        , 0.89267978, 0.80449101, 0.73068607, 0.66794344,
       0.61388555, 0.56677274, 0.52530997, 0.4885144 , 0.45562796,
       0.42605559, 0.39932347, 0.37504883, 0.85497712])

In [23]:
svc_table = service_table(demographic_tbl = demographic_table(table = mortality_base,
                  employee_gender = data.gender.iloc[0],
                  employee_age = data.age.iloc[0]))

In [24]:
svc_table_dec = np.multiply(svc_table.iloc[:-1,1:].values,np.vstack((np.vstack((financial_assumption[:-1],financial_assumption[:-1])),financial_assumption[:-1])).T)
svc_table_dec

array([[1.53428290e-03, 1.53194370e-05, 3.69130851e-03],
       [1.49559009e-03, 1.49308778e-05, 3.01665842e-03],
       [1.48422785e-03, 1.48148005e-05, 2.47055427e-03],
       [1.49958595e-03, 1.49648310e-05, 2.02103395e-03],
       [1.50158430e-03, 1.49816511e-05, 1.64596205e-03],
       [1.51117118e-03, 1.50738606e-05, 1.32946825e-03],
       [1.52070423e-03, 1.51653389e-05, 1.05995346e-03],
       [1.54505497e-03, 1.54038954e-05, 8.28716170e-04],
       [1.57118589e-03, 1.56597488e-05, 6.29139354e-04],
       [1.60264027e-03, 1.59680132e-05, 4.56071181e-04],
       [1.63381878e-03, 1.62730007e-05, 3.05441258e-04],
       [1.67188948e-03, 1.66457330e-05, 1.73973372e-04],
       [1.70796419e-03, 1.69979682e-05, 5.90062016e-05]])

In [25]:
ben_fac_employee = select_ben_fac(employee_age=data.age.iloc[0], employee_yos= data.yos.iloc[0],pension_age=pension_age)

In [27]:
dbo_exclude_pension = np.multiply(svc_table_dec,ben_fac_employee.values)*data.salary.iloc[0]
dbo_exclude_pension

array([[468904.15585047,   4681.89253565,  51278.61650661],
       [457078.94459952,   4563.14192893,  41906.56776653],
       [453606.44209404,   4527.66664784,  34320.2429591 ],
       [479131.97672038,   4781.40584027,  28075.63348593],
       [479770.46646072,   4786.78003982,  22865.24056104],
       [482833.56772935,   4816.24183895,  18468.59788376],
       [507004.66216408,   5056.14264765,  14724.57443484],
       [515123.22852544,   5135.67766495,  11512.2912623 ],
       [523835.30766058,   5220.97949661,   8739.82644248],
       [556585.66543703,   5545.57837917,   6335.61219386],
       [567413.73904134,   5651.49837211,   4243.10379436],
       [580635.42460029,   5780.94567803,   2416.78901197],
       [616890.47552912,   6139.40547861,    819.69750785]])

# IFRIC Implementation

Cut off age of IFRIC implementation is 56-24 = 32 years old. Therefore 

In [43]:
def ifric_cutoff(employee_age, employee_yos, pension_age):
    return int((pension_age - employee_age + employee_yos)/3)*3

In [46]:
def pension_ifric(employee_age, employee_yos, pension_age):
    return (employee_yos-(pension_age - employee_age + employee_yos-ifric_cutoff(employee_age, employee_yos, pension_age)))/ifric_cutoff(employee_age, employee_yos, pension_age)

In [58]:
def pension_pbo(employee_age, employee_yos, pension_age, financial_assumption, svc_table):
    return ben_fac.iloc[ifric_cutoff(employee_age, employee_yos, pension_age),0]*pension_ifric(employee_age, employee_yos, pension_age)*financial_assumption[-1]*svc_table.iloc[-1,0]

In [62]:
pension = pension_pbo(data.age.iloc[0], data.yos.iloc[0], pension_age, financial_assumption, svc_table)*data.salary.iloc[0]

# Total for Employee 1

In [63]:
pension + np.sum(dbo_exclude_pension)

113249973.56810696